In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from typing import Tuple, Optional, Dict
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
from nsdcode.nsd_mapdata import NSDmapdata

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from research.data.natural_scenes import (
    NaturalScenesDataset,
    StimulusDataset,
)

In [4]:
nsd_path = Path('D:\\Datasets\\NSD\\')
nsd = NaturalScenesDataset(nsd_path)
nsd_mapdata = NSDmapdata(nsd_path)
freesurfer_path = nsd_path / 'nsddata/freesurfer'
derivatives_path = nsd_path / 'derivatives'

In [72]:
# Map cortical parcellations to volume spaces
# Note: Had to make a slight change to nsdcode mapsurfacetovolume.py line 125:
# for i, c_label in enumerate(all_labels):
#     data_new[i, :] = data[data_q, :] == c_label

annot_files = ['aparc', 'aparc.a2009s', 'aparc.DKTatlas']
hemispheres = ['lh', 'rh', 'combine']

for annot_file in annot_files:
    print(annot_file)
    
    for hemi in hemispheres:
        print(hemi)
        
        for i in tqdm(range(1, 9)):
            subject_name = f'subj0{i}'
            label_path = freesurfer_path / subject_name / 'label'
            
            def load_hemi(hemi):
                hemi_path = label_path / f'{hemi}.{annot_file}.annot'
                hemi_data, hemi_ctab, hemi_names = fio.read_annot(hemi_path)
                hemi_data = hemi_data[:, None]
                hemi_source_data = [hemi_data] * 3
                hemi_source_space = [f'{hemi}.layerB1', f'{hemi}.layerB2', f'{hemi}.layerB3']
                return hemi_source_data, hemi_source_space, hemi_ctab, hemi_names
            
            if hemi == 'combine':
                file_name = f'{annot_file}.nii.gz'
                lh_source_data, lh_source_space, lh_ctab, lh_names = load_hemi('lh')
                rh_source_data, rh_source_space, rh_ctab, rh_names = load_hemi('rh')
                source_data = lh_source_data + rh_source_data
                source_space = lh_source_space + rh_source_space
                ctab = lh_ctab 
                names = lh_names
                ctab_file_name = f'{annot_file}.ctab'
            else:
                file_name = f'{hemi}.{annot_file}.nii.gz'
                source_data, source_space, ctab, names = load_hemi(hemi)
                ctab_file_name = f'{hemi}.{annot_file}.ctab'
                
            ctab_path = derivatives_path / 'labels' / subject_name
            ctab_path.mkdir(exist_ok=True, parents=True)
            with open(ctab_path / ctab_file_name, 'w') as f:
                for j, name in enumerate(names):
                    f.write(f'{j} {name.decode("utf-8")}\n')

            target_space = 'anat0pt8'
            output_path = derivatives_path / 'images' / subject_name / 'anat0pt8mm'
            output_path.mkdir(exist_ok=True, parents=True)
            output_file = output_path / file_name
            nsd_mapdata.fit(
                i, 
                source_space, 
                target_space, 
                source_data, 
                interptype='surfacewta', 
                badval=-1, 
                outputfile=output_file
            )

            source_data = str(output_file)
            for target_space, space_name in [('func1pt8', 'func1pt8mm'), ('func1pt0', 'func1mm')]:
                source_space = 'anat0pt8'
                output_path = derivatives_path / 'images' / subject_name / space_name
                output_path.mkdir(exist_ok=True, parents=True)
                output_file = output_path / file_name
                nsd_mapdata.fit(
                    i,
                    source_space,
                    target_space,
                    source_data,
                    interptype='wta',
                    badval=-1,
                    outputfile=output_file
                )

aparc
lh


  0%|          | 0/8 [00:00<?, ?it/s]

rh


  0%|          | 0/8 [00:00<?, ?it/s]

combine


...

  0%|          | 0/8 [00:00<?, ?it/s]

rh


  0%|          | 0/8 [00:00<?, ?it/s]

combine


  0%|          | 0/8 [00:00<?, ?it/s]

In [89]:
# map HCP MNI cortical atlas to subject spaces

hcp_path = derivatives_path / 'HCP-MMP1'
hcp_cortices_file_path = hcp_path / 'HCP-MMP1_cortices' / 'HCP-MMP1_cortices_1mm.nii.gz'
hcp_cortices_data = np.rint(nib.load(hcp_cortices_file_path).get_fdata()).astype(int)

hcp_cortices_merged = hcp_cortices_data.copy()
hcp_cortices_merged[hcp_cortices_merged >= 100] -= 100

hcp_cortices_lh = hcp_cortices_data.copy()
hcp_cortices_lh[hcp_cortices_lh >= 100] = 0

hcp_cortices_rh = hcp_cortices_data.copy()
hcp_cortices_rh[hcp_cortices_rh <= 100] = 0
hcp_cortices_rh[hcp_cortices_rh >= 100] -= 100

target_spaces = [
    ('func1pt8', 'func1pt8mm'), 
    ('func1pt0', 'func1mm'), 
    ('anat0pt8', 'anat0pt8mm')
]
source_space = 'MNI'
source_datas = [
    (hcp_cortices_merged, 'HCP_MMP1_cortices.nii.gz'),
    (hcp_cortices_lh, 'lh.HCP_MMP1_cortices.nii.gz'),
    (hcp_cortices_rh, 'rh.HCP_MMP1_cortices.nii.gz')
]

for i in tqdm(range(1, 9)):
    subject_name = f'subj0{i}'
    
    for target_space, space_name in target_spaces:
        output_path = derivatives_path / 'images' / subject_name / space_name
        output_path.mkdir(exist_ok=True, parents=True)
        
        for source_data, file_name in source_datas:
            output_file = output_path / file_name
            nsd_mapdata.fit(
                i, 
                source_space, 
                target_space, 
                source_data,
                interptype='nearest',
                badval=0, 
                outputfile=str(output_file)
            )

  0%|          | 0/8 [00:00<?, ?it/s]

data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array passed
data array

In [3]:
from bs4 import BeautifulSoup




NameError: name 'hcp_path' is not defined

In [6]:
from bs4 import BeautifulSoup

hcp_path = derivatives_path / 'HCP-MMP1'

with open(hcp_path / 'HCP-MMP1.xml') as f:
    hcp_xml = BeautifulSoup(f.read(), 'xml')
    
with open(hcp_path / 'HCP-MMP1_cortices.xml') as f:
    hcp_cortices_xml = BeautifulSoup(f.read(), 'xml')

hcp_cortices_file_path = hcp_path / 'HCP-MMP1_cortices' / 'HCP-MMP1_cortices_2mm.nii.gz'
hcp_cortices_data = np.rint(nib.load(hcp_cortices_file_path).get_fdata()).astype(int)
hcp_cortices_merged = hcp_cortices_data.copy()
hcp_cortices_merged[hcp_cortices_merged >= 100] -= 100

hcp_file_path = hcp_path / 'HCP-MMP1' / 'HCP-MMP1_2mm.nii.gz'
hcp_data = np.rint(nib.load(hcp_file_path).get_fdata()).astype(int)
hcp_merged = hcp_data.copy()
hcp_merged[hcp_merged >= 200] -= 200

hcp_names = {
    i: hcp_xml.find_all('label')[i].contents[0][:-2]
    for i in np.unique(hcp_merged)[1:]
}

hcp_cortices_names = {
    i: hcp_cortices_xml.find_all('label')[i].contents[0][:-2]
    for i in np.unique(hcp_cortices_merged)[1:]
}

hcp_ids = np.unique(hcp_merged)[1:]
hcp_cortices_ids = np.unique(hcp_cortices_merged)[1:]

hcp_names = {
    i: hcp_xml.find_all('label')[i].contents[0][:-2]
    for i in hcp_ids
}

hcp_cortices_names = {
    i: hcp_cortices_xml.find_all('label')[i].contents[0][:-2]
    for i in hcp_cortices_ids
}

corticies_id_mapping = {}
cortices_name_mapping = {}
for i in hcp_cortices_ids:
    intersection = np.unique((hcp_cortices_merged == i) * hcp_merged)[1:]
    corticies_id_mapping[i] = intersection
    
    cortices_name = hcp_cortices_names[i]
    cortices_name_mapping[cortices_name] = [hcp_names[j] for j in intersection]

for k, v in cortices_name_mapping.items():
    print(k, v)

Primary_Visual ['V1']
Early_Visual ['V2', 'V3', 'V4']
Dorsal_Stream_Visual ['V6', 'V3A', 'V7', 'IPS1', 'V3B', 'V6A']
Ventral_Stream_Visual ['V8', 'FFC', 'PIT', 'VMV1', 'VMV3', 'VMV2', 'VVC']
MT+_Complex_and_Neighboring_Visual_Areas ['MST', 'LO1', 'LO2', 'MT', 'PH', 'V4t', 'FST', 'V3CD', 'LO3']
Somatosensory_and_Motor ['4', '3b', '1', '2', '3a']
Paracentral_Lobular_and_Mid_Cingulate ['5m', '5mv', '23c', '5L', '24dd', '24dv', 'SCEF', '6ma', '6mp']
Premotor ['FEF', 'PEF', '55b', '6d', '6v', '6r', '6a']
Posterior_Opercular ['43', 'OP4', 'OP1', 'OP2-3', 'FOP1']
Early_Auditory ['A1', '52', 'RI', 'PFcm', 'PBelt', 'MBelt', 'LBelt']
Auditory_Association ['TA2', 'STGa', 'A5', 'STSda', 'STSdp', 'STSvp', 'A4', 'STSva']
Insular_and_Frontal_Opercular ['PoI2', 'FOP4', 'MI', 'Pir', 'AVI', 'AAIC', 'FOP3', 'FOP2', 'PoI1', 'Ig', 'FOP5', 'PI']
Medial_Temporal ['EC', 'PreS', 'H', 'PeEc', 'PHA1', 'PHA3', 'TF', 'PHA2']
Lateral_Temporal ['TGd', 'TE1a', 'TE1p', 'TE2a', 'TE2p', 'PHT', 'TGv', 'TE1m']
Temporo-Par

In [23]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

ctab_text = '\n'.join([f'{name} {i}'for name, i in zip(corticies_id_mapping.keys(), cortices_name_mapping.keys())])
ctab_text = '0 Unknown\n' + ctab_text

for subject in nsd.subjects.keys():
    for hemi in ('', 'lh.', 'rh.'):
        with open(derivatives_path / f'labels/{subject}/{hemi}HCP_MMP1_cortices.ctab', 'w') as f:
            f.write(ctab_text)
        
        hcp_roi, _ = nsd.load_roi(subject, f'{hemi}HCP_MMP1')
        hcp_cortices_roi = np.zeros_like(hcp_roi)
        
        for cortices_id, region_ids in corticies_id_mapping.items():
            for region_id in region_ids:
                hcp_cortices_roi[hcp_roi == region_id] = cortices_id
                
        affine = nsd.get_affine(subject)
        
        nib.save(
            nib.Nifti1Image(hcp_cortices_roi, affine), 
            derivatives_path / f'images/{subject}/func1pt8mm/{hemi}HCP_MMP1_cortices.nii.gz'
        )

In [22]:

print(ctab_text)

0 Unknown
1 Primary_Visual
2 Early_Visual
3 Dorsal_Stream_Visual
4 Ventral_Stream_Visual
5 MT+_Complex_and_Neighboring_Visual_Areas
6 Somatosensory_and_Motor
7 Paracentral_Lobular_and_Mid_Cingulate
8 Premotor
9 Posterior_Opercular
10 Early_Auditory
11 Auditory_Association
12 Insular_and_Frontal_Opercular
13 Medial_Temporal
14 Lateral_Temporal
15 Temporo-Parieto-Occipital_Junction
16 Superior_Parietal
17 Inferior_Parietal
18 Posterior_Cingulate
19 Anterior_Cingulate_and_Medial_Prefrontal
20 Orbital_and_Polar_Frontal
21 Inferior_Frontal
22 Dorsolateral_Prefrontal
